In [1]:
## 1. Data loading

import os
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_PATH = os.path.join('..', 'data')
FILENAME = 'yolo.pdf'
document_path = os.path.join(DATA_PATH, FILENAME)

loader = PyPDFLoader(document_path)
docs = loader.load()

print('Documents length - ', len(docs))
document_text = docs[0].page_content
print('page content - ', document_text)


## 2. Chunking
recursive_char_text_splitter=RecursiveCharacterTextSplitter(
                                                chunk_size=500,
                                                chunk_overlap=50)
documents=recursive_char_text_splitter.split_documents(docs)

Documents length -  10
page content -  You Only Look Once:
Uniﬁed, Real-Time Object Detection
Joseph Redmon∗, Santosh Divvala∗†, Ross Girshick¶, Ali Farhadi∗†
University of Washington∗, Allen Institute for AI†, Facebook AI Research¶
http://pjreddie.com/yolo/
Abstract
We present YOLO, a new approach to object detection.
Prior work on object detection repurposes classiﬁers to per-
form detection. Instead, we frame object detection as a re-
gression problem to spatially separated bounding boxes and
associated class probabilities. A single neural network pre-
dicts bounding boxes and class probabilities directly from
full images in one evaluation. Since the whole detection
pipeline is a single network, it can be optimized end-to-end
directly on detection performance.
Our uniﬁed architecture is extremely fast. Our base
YOLO model processes images in real-time at 45 frames
per second. A smaller version of the network, Fast YOLO,
processes an astounding 155 frames per second while
still achie

In [2]:
## 3. Embeddings - huggingface Embeddings

from langchain.embeddings import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(
                model_name='sentence-transformers/all-MiniLM-L6-v2', 
                model_kwargs={'device':'cpu'}
        )

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## 3. Save Embeddings in vectorDB - FAISS

FAISS_VECTORDB_PATH = os.path.join('vectordb', 'faiss_vector_db')

vector_db_directory = FAISS_VECTORDB_PATH
from langchain.vectorstores import FAISS

def store_data_in_vectordb(documents, embeddings):
    # try:
    #     current_vectordb = load_vectordb(vector_db_directory, embeddings)
    #     print('current_vectordb - ', current_vectordb)
    # except Exception as e:
    #     print('Exception inside storing data in vector db => ', e)

    new_knowledge_base =FAISS.from_documents(documents, embeddings)
    print('new_knowledge_base - ', new_knowledge_base)

    # Saving the new vector DB
    new_knowledge_base.save_local(vector_db_directory)
    return new_knowledge_base


def load_vectordb(stored_directory, embeddings):
    loaded_vector_db = FAISS.load_local(stored_directory, embeddings, allow_dangerous_deserialization=True)
    return loaded_vector_db

store_data_in_vectordb(documents, embeddings) # storing data into vectorDB
vector_db = load_vectordb(vector_db_directory, embeddings) # reading data from vectordb database

retriever_faissdb = vector_db.as_retriever(search_kwargs={"k": 2})




new_knowledge_base -  <langchain_community.vectorstores.faiss.FAISS object at 0x000001CBEEAEA9D0>


In [5]:
# View data in vector DB

import pandas as pd
def convert_vectordb_to_df(vectorDb):
    try:
        vector_dict = vectorDb.docstore._dict
        data_rows = []

        for k in vector_dict.keys():
            doc_name = vector_dict[k].metadata['source'].split('/')[-1]
            if 'page' not in vector_dict[k].metadata:
                page_number = 1
            else:
                page_number = vector_dict[k].metadata['page'] + 1
            content =  vector_dict[k].page_content
            data_rows.append({"chunk_id": k, "document": doc_name, "page": page_number, "content":content})

        vector_df = pd.DataFrame(data_rows)
        return vector_df
    except Exception as e :
        print('Error in convert_vectordb_to_df - ', e)
        return None
    

convert_vectordb_to_df(vector_db) # for faissdb

,chunk_id,document,page,content
0,d5a5a571-cd30-4b88-8651-85030c7e193a,..\data\yolo.pdf,1,"You Only Look Once:\nUniﬁed, Real-Time Object ..."
1,787504a2-d594-4ba9-875c-45f99682b85b,..\data\yolo.pdf,1,associated class probabilities. A single neura...
2,9f64f170-2569-4e75-b928-529c40e19b11,..\data\yolo.pdf,1,still achieving double the mAP of other real-t...
3,d5636b5f-f6c8-4b54-bac3-ac4fbd8fc784,..\data\yolo.pdf,1,"jects are in the image, where they are, and ho..."
4,4da755aa-ab4e-43bd-9130-cd51fee17583,..\data\yolo.pdf,1,"for general purpose, responsive robotic system..."
...,...,...,...,...
91,d4c8fa44-4413-4361-a2ba-479bc64c5902,..\data\yolo.pdf,9,"abs/1504.06066, 2015. 3, 7\n[30] O. Russakovsk..."
92,2cd73001-8612-4274-a4ef-55ce336bd3d3,..\data\yolo.pdf,9,and Y . LeCun. Overfeat: Integrated recognitio...
93,00e57314-d73d-4c7d-aba7-d61cba180fac,..\data\yolo.pdf,10,[33] Z. Shen and X. Xue. Do more dropouts in p...
94,97798490-d18d-4502-ba79-565d5bad9142,..\data\yolo.pdf,10,4\n[36] P. Viola and M. Jones. Robust real-tim...


In [6]:
def get_similiar_docs(vector_db, query,k=3,score=False):
  
    vectordb_results = list()
    if score:
        similar_docs_with_score = vector_db.similarity_search_with_score(query,k=k)
        for doc, score in similar_docs_with_score:
            print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")
            vectordb_results.append(doc.page_content)
    else:
        similar_docs = vector_db.similarity_search(query,k=k)
        for doc in similar_docs:
            vectordb_results.append(doc.page_content)

    return vectordb_results


get_similiar_docs(vector_db, 'what is YOLO?')

['same between YOLO and Fast YOLO.',
 'Figure 6: Qualitative Results. YOLO running on sample artwork and natural images from the internet. It is mostly accurate although it\ndoes think one person is an airplane.\nincluding the time to fetch images from the camera and dis-\nplay the detections.\nThe resulting system is interactive and engaging. While\nYOLO processes images individually, when attached to a\nwebcam it functions like a tracking system, detecting ob-\njects as they move around and change in appearance. A',
 'making predictions. Unlike sliding window and region\nproposal-based techniques, YOLO sees the entire image\nduring training and test time so it implicitly encodes contex-\ntual information about classes as well as their appearance.\nFast R-CNN, a top detection method [14], mistakes back-\nground patches in an image for objects because it can’t see\nthe larger context. YOLO makes less than half the number\nof background errors compared to Fast R-CNN.']

In [7]:

## prompt templates:

from langchain import PromptTemplate
from langchain.schema.prompt_template import format_document

llm_prompt_template = """You are an assistant for question-answering tasks.
Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences minimum and show the answer in bullet points and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks.\nUse the following context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences minimum and show the answer in bullet points and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:"


In [8]:
from langchain import HuggingFaceHub


HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN


llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2", huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [9]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=retriever_faissdb,
            return_source_documents=True,
            chain_type_kwargs={'prompt': llm_prompt}
    )

result=chain({'query':'What is YOLO?'}, return_only_outputs=True)
print(result['result'])

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


You are an assistant for question-answering tasks.
Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences minimum and show the answer in bullet points and keep the answer concise.

Question: What is YOLO? 
Context: same between YOLO and Fast YOLO.

Figure 6: Qualitative Results. YOLO running on sample artwork and natural images from the internet. It is mostly accurate although it
does think one person is an airplane.
including the time to fetch images from the camera and dis-
play the detections.
The resulting system is interactive and engaging. While
YOLO processes images individually, when attached to a
webcam it functions like a tracking system, detecting ob-
jects as they move around and change in appearance. A 
Answer:
- YOLO stands for You Only Look Once, which is a real-time object detection system.
- It was developed by Joseph Redmon and Ali Farhadi in 2015.
- YOLO uses a single neural network to predict b